In [1]:
import gzip
import os
import sys
import urllib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image

import code
import os
#import tensorflow.python.platform

import numpy as np
import tensorflow.keras as keras
from keras import layers, models, datasets,regularizers, Model, Input
from keras.layers import concatenate, Conv2D, Conv2DTranspose, BatchNormalization, MaxPooling2D, Dropout
from helpers_images import * 


In [2]:
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

GPU found


In [3]:
NUMBER_TRAINING_EXAMPLES = 100
NUMBER_TEST_EXAMPLES = 50
DATA_LOADED = False

In [4]:
training_data_directory = "../data/training/"
test_data_directory = "../data/test_set_images/"
if not(DATA_LOADED):    
    train_data_orig = extract_train_data(NUMBER_TRAINING_EXAMPLES)
    train_labels_orig =  extract_labels(training_data_directory, NUMBER_TRAINING_EXAMPLES,True)
    DATA_LOADED = True

print(train_data_orig.shape)
print(train_labels_orig.shape)

Loading ../data/training/images/satImage_001.png
Loading ../data/training/images/satImage_002.png
Loading ../data/training/images/satImage_003.png
Loading ../data/training/images/satImage_004.png
Loading ../data/training/images/satImage_005.png
Loading ../data/training/images/satImage_006.png
Loading ../data/training/images/satImage_007.png
File ../data/training/images/satImage_008.png does not exist
Loading ../data/training/images/satImage_009.png
Loading ../data/training/images/satImage_010.png
File ../data/training/images/satImage_011.png does not exist
Loading ../data/training/images/satImage_012.png
Loading ../data/training/images/satImage_013.png
Loading ../data/training/images/satImage_014.png
Loading ../data/training/images/satImage_015.png
Loading ../data/training/images/satImage_016.png
Loading ../data/training/images/satImage_017.png
Loading ../data/training/images/satImage_018.png
Loading ../data/training/images/satImage_019.png
Loading ../data/training/images/satImage_020.

Loading ../data/training/groundtruth/satImage_066.png
Loading ../data/training/groundtruth/satImage_067.png
Loading ../data/training/groundtruth/satImage_068.png
Loading ../data/training/groundtruth/satImage_069.png
Loading ../data/training/groundtruth/satImage_070.png
Loading ../data/training/groundtruth/satImage_071.png
Loading ../data/training/groundtruth/satImage_072.png
Loading ../data/training/groundtruth/satImage_073.png
Loading ../data/training/groundtruth/satImage_074.png
Loading ../data/training/groundtruth/satImage_075.png
Loading ../data/training/groundtruth/satImage_076.png
Loading ../data/training/groundtruth/satImage_077.png
File ../data/training/groundtruth/satImage_078.png does not exist
Loading ../data/training/groundtruth/satImage_079.png
Loading ../data/training/groundtruth/satImage_080.png
Loading ../data/training/groundtruth/satImage_081.png
Loading ../data/training/groundtruth/satImage_082.png
Loading ../data/training/groundtruth/satImage_083.png
Loading ../data/

In [5]:
#plt.imshow(train_data_orig[187])

In [6]:
IMG_PATCH_SIZE = 400
train_data = train_data_orig
train_labels = train_labels_orig

In [7]:
def encoding_block(input_, nbr_filters = 16, pooling_ = True, dropout = True):
    conv = Conv2D(nbr_filters, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.0001), kernel_initializer='he_normal', padding='same') (input_)
    conv = BatchNormalization()(conv)
    if dropout:
        conv = Dropout(0.1) (conv)
    conv = Conv2D(nbr_filters, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.0001), kernel_initializer='he_normal', padding='same') (conv)
    conv = BatchNormalization()(conv)
    if pooling_:
        pooling = MaxPooling2D((2,2))(conv)
        return conv, pooling
    return conv

In [8]:
def decoding_block(conv, previous_conv,nbr_filters = 16, dropout = True):
    upsample = Conv2DTranspose(nbr_filters, (2, 2),kernel_regularizer=regularizers.l2(0.01), strides=(2, 2), padding='same') (conv)
    upsample = concatenate([upsample, previous_conv])
    conv = Conv2D(nbr_filters, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.0001), kernel_initializer='he_normal', padding='same') (upsample)
    conv = BatchNormalization() (conv)
    if dropout:
        conv = Dropout(0.2) (conv)
    conv = Conv2D(nbr_filters, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.0001), kernel_initializer='he_normal', padding='same') (conv)
    conv = BatchNormalization() (conv)
    return conv

In [9]:
def create_custom_unet(nbr_filters = 16, dropout = True):
    inputs = Input((IMG_PATCH_SIZE, IMG_PATCH_SIZE, 3))
    conv1, pooling1 = encoding_block(inputs, nbr_filters, True, dropout)
    conv2, pooling2 = encoding_block(pooling1, nbr_filters*2, True,dropout)
    conv3, pooling3 = encoding_block(pooling2, nbr_filters*4, True,dropout)
    conv4, pooling4 = encoding_block(pooling3, nbr_filters*8, True,dropout)
    conv5 = encoding_block(pooling4, nbr_filters*16,False, dropout)

    conv6 = decoding_block(conv5, conv4, nbr_filters*8, dropout)
    conv7 = decoding_block(conv6, conv3, nbr_filters*4, dropout)
    conv8 = decoding_block(conv7, conv2, nbr_filters*2, dropout)
    conv9 = decoding_block(conv8, conv1, nbr_filters, dropout)
    outputs = Conv2D(1, (1, 1), activation='sigmoid') (conv9)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model


In [10]:
from keras import backend as K

def dice_coef(y_true, y_pred, smooth = 1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def soft_dice_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [11]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystopper = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 10,
                          verbose = 1,
                          mode = "min",
                          restore_best_weights = True)
lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               mode="min",
                               min_delta=1e-4)

In [12]:
model = create_custom_unet(16, True)
model.summary()
model.compile(optimizer='adam',
            loss=soft_dice_loss,
          metrics=[dice_coef])       

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 400, 400, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 400, 400, 16  448         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 400, 400, 16  64         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [13]:
epochs = 200
history = model.fit(train_data, train_labels, epochs=epochs, validation_split = 0.05, batch_size = 8,callbacks = [ earlystopper, lr_reducer])

Epoch 1/200
110/110 [==============================] - 17s 93ms/step - loss: 1.3322 - dice_coef: 0.5569 - val_loss: 0.6698 - val_dice_coef: 0.5261 - lr: 0.0010
Epoch 2/200
110/110 [==============================] - 8s 76ms/step - loss: 0.4515 - dice_coef: 0.6972 - val_loss: 0.3700 - val_dice_coef: 0.7554 - lr: 0.0010
Epoch 3/200
110/110 [==============================] - 8s 76ms/step - loss: 0.3625 - dice_coef: 0.7572 - val_loss: 0.3999 - val_dice_coef: 0.7140 - lr: 0.0010
Epoch 4/200
110/110 [==============================] - 8s 76ms/step - loss: 0.3266 - dice_coef: 0.7850 - val_loss: 0.3333 - val_dice_coef: 0.7767 - lr: 0.0010
Epoch 5/200
110/110 [==============================] - 8s 76ms/step - loss: 0.2918 - dice_coef: 0.8123 - val_loss: 0.2449 - val_dice_coef: 0.8562 - lr: 0.0010
Epoch 6/200
110/110 [==============================] - 8s 76ms/step - loss: 0.2768 - dice_coef: 0.8228 - val_loss: 0.2543 - val_dice_coef: 0.8431 - lr: 0.0010
Epoch 7/200
110/110 [========================

110/110 [==============================] - 8s 76ms/step - loss: 0.1215 - dice_coef: 0.9256 - val_loss: 0.1012 - val_dice_coef: 0.9456 - lr: 1.0000e-04
Epoch 51/200
110/110 [==============================] - 8s 76ms/step - loss: 0.1194 - dice_coef: 0.9270 - val_loss: 0.1008 - val_dice_coef: 0.9454 - lr: 1.0000e-04
Epoch 52/200
110/110 [==============================] - 8s 76ms/step - loss: 0.1193 - dice_coef: 0.9261 - val_loss: 0.1004 - val_dice_coef: 0.9454 - lr: 1.0000e-04
Epoch 53/200
110/110 [==============================] - 8s 76ms/step - loss: 0.1209 - dice_coef: 0.9235 - val_loss: 0.0998 - val_dice_coef: 0.9455 - lr: 1.0000e-04
Epoch 54/200
110/110 [==============================] - 8s 76ms/step - loss: 0.1187 - dice_coef: 0.9267 - val_loss: 0.0988 - val_dice_coef: 0.9463 - lr: 1.0000e-04
Epoch 55/200
110/110 [==============================] - 8s 77ms/step - loss: 0.1172 - dice_coef: 0.9275 - val_loss: 0.0971 - val_dice_coef: 0.9473 - lr: 1.0000e-04
Epoch 56/200
110/110 [=======

110/110 [==============================] - 9s 78ms/step - loss: 0.0951 - dice_coef: 0.9363 - val_loss: 0.0807 - val_dice_coef: 0.9509 - lr: 1.0000e-04
Epoch 101/200
110/110 [==============================] - 9s 78ms/step - loss: 0.0931 - dice_coef: 0.9382 - val_loss: 0.0790 - val_dice_coef: 0.9522 - lr: 1.0000e-04
Epoch 102/200
110/110 [==============================] - 9s 78ms/step - loss: 0.0921 - dice_coef: 0.9390 - val_loss: 0.0789 - val_dice_coef: 0.9520 - lr: 1.0000e-04
Epoch 103/200
110/110 [==============================] - 9s 78ms/step - loss: 0.0921 - dice_coef: 0.9387 - val_loss: 0.0789 - val_dice_coef: 0.9519 - lr: 1.0000e-04
Epoch 104/200
110/110 [==============================] - 8s 76ms/step - loss: 0.0912 - dice_coef: 0.9393 - val_loss: 0.0777 - val_dice_coef: 0.9528 - lr: 1.0000e-04
Epoch 105/200
110/110 [==============================] - 8s 77ms/step - loss: 0.0921 - dice_coef: 0.9381 - val_loss: 0.0860 - val_dice_coef: 0.9448 - lr: 1.0000e-04
Epoch 106/200
110/110 [=

Epoch 145/200
110/110 [==============================] - 9s 80ms/step - loss: 0.0859 - dice_coef: 0.9434 - val_loss: 0.0750 - val_dice_coef: 0.9544 - lr: 1.0000e-09
Epoch 146/200
110/110 [==============================] - 9s 81ms/step - loss: 0.0854 - dice_coef: 0.9438 - val_loss: 0.0750 - val_dice_coef: 0.9544 - lr: 1.0000e-09
Epoch 147/200
110/110 [==============================] - 9s 78ms/step - loss: 0.0857 - dice_coef: 0.9435 - val_loss: 0.0749 - val_dice_coef: 0.9544 - lr: 1.0000e-09
Epoch 148/200
109/110 [============================>.] - ETA: 0s - loss: 0.0859 - dice_coef: 0.9434
Epoch 00148: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-10.
110/110 [==============================] - 9s 83ms/step - loss: 0.0859 - dice_coef: 0.9432 - val_loss: 0.0750 - val_dice_coef: 0.9544 - lr: 1.0000e-09
Epoch 149/200
110/110 [==============================] - 9s 85ms/step - loss: 0.0857 - dice_coef: 0.9436 - val_loss: 0.0750 - val_dice_coef: 0.9544 - lr: 1.0000e-10
Epoch 150

In [14]:
import pathlib
directory_path = pathlib.Path(test_data_directory)

In [15]:
all_images = []
for i in range(1,51):
    image_in_directory = list(directory_path.glob("test_" + str(i) + "/*"))
    all_images.extend(image_in_directory)
img_height = img_width = 608

In [16]:
all_images_arrays = []
for i in range(len(all_images)):
    img = tf.keras.utils.load_img(
        all_images[i], target_size=(img_height, img_width)
    )
    img_array = tf.keras.utils.img_to_array(img)
    all_images_arrays.append(img_array)
all_images_arrays = numpy.array(all_images_arrays)/255.0

In [17]:
predictions_testing = "./predictions/"
if not os.path.isdir(predictions_testing):
    os.mkdir(predictions_testing)

In [18]:
def predict(k):
    img = all_images_arrays[k]
    extract_1 = img[:400, :400]
    extract_2 = img[:400, -400:]
    extract_3 = img[-400:, :400]
    extract_4 = img[-400:, -400:]
    output_prediction = []
    output_prediction.append(model(np.expand_dims(extract_1 , axis=0)).numpy() )
    output_prediction.append(model(np.expand_dims(extract_2 , axis=0)).numpy() )
    output_prediction.append(model(np.expand_dims(extract_3 , axis=0)).numpy() )
    output_prediction.append(model(np.expand_dims(extract_4 , axis=0)).numpy() )

    output_prediction[0] = output_prediction[0].reshape(400,400)
    output_prediction[1] = output_prediction[1].reshape(400,400)
    output_prediction[2] = output_prediction[2].reshape(400,400)
    output_prediction[3] = output_prediction[3].reshape(400,400)
    gt_img = numpy.zeros([img.shape[0], img.shape[1]])
    gt_img[:400,:400] = output_prediction[0]
    gt_img[:400, -400:] = output_prediction[1]
    gt_img[-400: , :400] = output_prediction[2]
    gt_img[-400: , -400:] = output_prediction[3]
    oimg = make_img_overlay(img, gt_img)
    gt_img8 = img_float_to_uint8(gt_img)          
    gt_img_3c = numpy.zeros((img.shape[0], img.shape[1], 3), dtype=numpy.uint8)
    gt_img_3c[:, :, 0] = gt_img8
    gt_img_3c[:, :, 1] = gt_img8
    gt_img_3c[:, :, 2] = gt_img8
    overlay = make_img_overlay(img, gt_img)
    Image.fromarray(gt_img_3c).save(predictions_testing + "prediction_image_" + str(k+1) + ".png")
    oimg.save(predictions_testing  + "overlay_" + str(k) + ".png")

In [19]:
for i in range(len(all_images_arrays)):
    predict(i)

In [20]:
5

5

In [22]:
model.save("./drive/MyDrive/Projet2ML/testing/model_possibly_great.h5")